## **Permutation Testing**

- For dominant meaning SCD

In [ ]:
import pandas as pd
import panel as pn
from tqdm.notebook import tqdm
from itertools import chain
import os
from collections import Counter
import numpy as np
import math

from scipy.spatial.distance import pdist, cosine
from scipy import stats
import random

import multiprocessing as mp
cpu_count = mp.cpu_count() - 4

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
sns.set(style='whitegrid', context='notebook')
plt.rcParams["font.family"] = "Latin Modern Sans"
fs = 14

# Adding custom colors
teal = "#60968a"
coral = "#ff6f61"
extended_palette = sns.color_palette("muted") + [teal, coral]


journals = ["pr", "pra", "prb", "prc", "prd", "pre", "prl", "rmp"]
years = [year for year in range(1924, 2023)]
basepath = "../data/embeddings/virtual_fulltexts/"

journal_dict = {"pr" : "Series II",
               "rmp" : "RMP",
               "prl" : "Letters", 
               "pra" : "PR - A",
               "prb" : "PR - B",
               "prc" : "PR - C",
               "prd" : "PR - D",
               "pre" : "PR - E"}

teal = "#60968a"
coral = "#ff6f61"
extended_palette = sns.color_palette("deep") + [teal, coral]
journal_colors = {"pr" : extended_palette[0],
               "rmp" : extended_palette[7],
               "prl" : sns.color_palette("muted")[5],
               "pra" : sns.color_palette("muted")[4],
               "prb" : extended_palette[1],
               "prc" : extended_palette[3],
               "prd" : extended_palette[2],
               "pre" : extended_palette[9]}

target_word = "virtual"

### **Load data**

In [ ]:
#df = pd.read_pickle("../data/cleaned_texts/virtual_fulltexts.df")

# create df with one occurrence / embedding as row
#wem_df = df.explode("deps")

# load embedding df:
wem_df = pd.read_pickle("../data/embeddings/virtual_fulltexts_virtual_token_embeddings.pkl")
# select only "virtual"
wem_df = wem_df.loc[wem_df.token == "virtual"]
wem_df = wem_df.drop("sentence_emb", axis=1)

wem_df.head(1)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24,8))

year_cut = 1948

wem_df.loc[wem_df.year <= year_cut].year.value_counts().sort_index().plot(ax = ax[0], kind="bar")

for index, value in enumerate(wem_df.loc[wem_df.year <= year_cut].year.value_counts().sort_index()):
    ax[0].text(index, value + 0.5, str(value), ha='center', va='bottom')


wem_df.loc[wem_df.year > year_cut].year.value_counts().sort_index().plot(ax = ax[1], kind="bar")

plt.tight_layout()
plt.suptitle("Embeddings per year for 'virtual'", fontweight="bold", y=1.01, fontsize=18)
plt.savefig("../results/permutation_testing/virtual_wem_count.png", bbox_inches="tight")
plt.show()

In [ ]:
def load_prt(target_word, emb_type, journal, slice_width):
    prt = pd.read_pickle(f"../data/scd/fb/prt_{target_word}/{target_word}_prt_fb_{emb_type}_{journal if journal else 'all'}_{slice_width}.pkl")
    prt =  make_ints(prt.T).T
    return prt

def load_jsd(target_word, emb_type, journal, cluster_type, slice_width):
    jsd = pd.read_pickle(f"../data/scd/sb/jsd/{target_word}_jsd_sb_{cluster_type}_{emb_type}_{journal if journal else 'all'}_{slice_width}.pkl")
    jsd =  make_ints(jsd.T).T
    return jsd
    
def make_ints(temp_df):
    if "-" in str(temp_df.columns[0]):
        temp_df.columns = [int(c.split("-")[0]) for c in temp_df.columns]
    if "_" in str(temp_df.columns[0]):
        temp_df.columns = [int(c.split("_")[0]) for c in temp_df.columns]
    return temp_df

In [ ]:
target_word = "virtual"
emb_type = "token_emb"
journal = None
slice_width = 1

fig, ax = plt.subplots(2, 1, figsize=(16,8), sharex=True)

prt = load_prt(target_word, emb_type, journal, slice_width)
prt.plot(ax = ax[0], lw=2.2)
ax[0].legend(["PRT"], loc="upper left")

jsd = load_jsd(target_word, emb_type, journal, cluster_type, slice_width)
jsd.plot(ax = ax[1], lw=2.2, label="JSD")
ax[1].legend(["JSD (k_means)"], loc="upper left")

plt.tight_layout()
plt.suptitle(f"SCD - Dominant Meaning for '{target_word}'", y=1.01, fontweight="bold")
plt.savefig("../results/permutation_testing/dom_meaning_plot.png", bbox_inches="tight")
plt.show()

### **Permutation Testing**

In [ ]:
# Calculate cosine distance between t1 and t2 embeddings
# input: tuple(t1_embeddings, t2_embeddings)
def get_cosine(wems, kind):
    t1_wem = get_mean_emb(wems[0])
    t2_wem = get_mean_emb(wems[1])
    if kind == "dist":
        return cosine(t1_wem, t2_wem)
    if kind == "sim":
        return 1 - cosine(t1_wem, t2_wem)
    if kind == "inverted":
        return 1 / (1 - cosine(t1_wem, t2_wem))

# create prototype embedding for sample
def get_mean_emb(embeddings):
    return np.mean(embeddings, axis=0)

# returns target metrics for t
# size of t1 and t2 remains intact
# add real values (s1 and s2), so that p is at least 1/r+1
# np.random.permutation = sampling without replacement
# r = sample size (iterations)
def get_permutations(s1, s2, r):

    # length = number of embeddings in smaller sample
    sample = s1 + s2
    size_t1 = len(s1)
    size_t2 = len(s2)
    # put in original real observation
    cosine_distances = [get_cosine([s1, s2], kind="inverted")]
    #aai_distances = [get_aaid([s1, s2])]
    #aid_differences = [get_aid_diff([s1, s2])]
    for n in range(r):
        samples_perm = np.random.permutation(sample)
        t1_perms = samples_perm[:size_t1]
        t2_perms = samples_perm[size_t1:]
        cosine_distances.append(get_cosine([t1_perms, t2_perms], kind="inverted"))
        #aai_distances.append(get_aaid([t1_perms, t2_perms]))
        #aid_differences.append(get_aid_diff([t1_perms, t2_perms]))

    return cosine_distances #aid_differences #aai_distances


# Choose side to test on (left, right, both)
# Choose metric (prt, aaid)
# Null hypothesis = average semantic change (or no semantic change?)
def get_p_value(dist, year, kind, tail):
    # load original value
    if kind == "prt":
        actual = prt.loc[year].values[0]
    elif kind == "aaid":
        actual = aaid.loc[year].values[0]
    else:
        return None
    
    # tail (left, right, both)
    if tail == "right":
        p_right = len([x for x in dist if x >= actual]) / len(dist)
        return p_right
    if tail == "left":
        p_left = len([x for x in dist if x <= actual]) / len(dist)
        return p_left
    if tail == "both":
        p_left = len([x for x in dist if x <= actual]) / len(dist)
        p_right = len([x for x in dist if x >= actual]) / len(dist)
        #return p_left + p_right

In [ ]:
r = 99999

for journal in tqdm(journals):

    # Load original PRT-values
    prt = load_prt(target_word="virtual", 
                    emb_type = "token_emb", 
                    journal = journal,  # None = all
                    slice_width = 1)

    data_df = wem_df.loc[wem_df.journal == journal]


    year_start = data_df.year.min()
    year_end = data_df.year.max()


    for year in tqdm(range(year_start+1, year_end+1)):

        if os.path.isfile(f"saves/perm_prt_cdists_{journal}_r={r+1}"):

            c_df = pd.read_pickle(f"saves/perm_prt_cdists_{journal}_r={r+1}")
            if year in c_df.columns:
                continue
        else:
            c_df = pd.DataFrame()
        cosine_distances = get_permutations(data_df.loc[data_df.year==year-1].token_emb.to_list(), 
                                data_df.loc[data_df.year==year].token_emb.to_list(), 
                                r=r)

        c_df[year] = cosine_distances
        c_df.to_pickle(f"saves/perm_prt_cdists_{journal}_r={r+1}")

In [ ]:
year_start = c_df.columns.min()
year_end = c_df.columns.max()

fig, ax = plt.subplots(math.ceil((year_end - year_start) / 4),4, figsize=(24, year_end - year_start), sharex=False, sharey=False)

year = year_start
for i in range(math.ceil((year_end - year_start) / 4)):
    ax[i][0].set_ylabel("Frequency")
    for j in range(4):

        if year not in c_df.columns:
            continue

        ax[i][j].hist(c_df[year],bins=50, density=False, edgecolor='black', linewidth=0.8)

        p_left = round(get_p_value(c_df[year], year, kind='prt', tail='left'), 4)
        p_right = round(get_p_value(c_df[year], year, kind='prt', tail='right'), 4)
        ax[i][j].set_title(f"{year-1} - {year} (p_left = {p_left}, p_right = {p_right})", fontweight="bold")

        ax[i][j].vlines(prt.loc[year], 0, ax[i][j].get_ylim()[1], color="red", lw=2.2)
        ax[i][j].xaxis.set_tick_params(labelbottom=True)
        year += 1

plt.suptitle(f"PRT distribution for {year_start-1} to {year_end} (r = {r+1})", fontweight="bold", fontsize=18, y=1.001)
plt.tight_layout()
plt.savefig(f"../results/permutation_testing/distribution_prt_{year_start-1}_{year_end}_r={r+1}.png", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,6), sharex=True)

fs=14

prt = load_prt(target_word, emb_type, journal, slice_width)
prt.plot(ax = ax, lw=2.5)
ax.legend(["PRT"], loc="upper left", fontsize=fs)
ax.set_ylabel("PRT", fontsize=fs+2)

ax2 = ax.twinx()
p_values = [get_p_value(c_df[year], year, kind="prt", tail="right") for year in [x for x in range(year_start, year_end)]]
ax2.scatter([x for x in range(year_start, year_end)], p_values, color=sns.color_palette()[3], lw=1.5)
ax2.grid(visible=False)
ax2.set_xlim(1920)

plt.axhline(0.05, color=sns.color_palette()[3], ls="--")
plt.axhline(0.95, color=sns.color_palette()[3], ls="--")
ax2.annotate("significance threshold semantic change (0.05)", (1999, 0.05), xytext=(1999, 0.051),
            fontsize=fs, color=sns.color_palette()[3], ha='left', va='bottom')
ax2.annotate("significance threshold no semantic change (0.95)", (1997, 0.95), xytext=(1997, 0.91),
            fontsize=fs, color=sns.color_palette()[3], ha='left', va='bottom')
            
#ax2.set_ylim(-.1, 1)
ax2.legend(["p-value"], fontsize=fs)
ax2.set_ylabel("p-value", fontsize=fs+2, color=sns.color_palette()[3])
ax2.tick_params(axis='y', colors=sns.color_palette()[3])

plt.tight_layout()
plt.suptitle(f"SCD - Dominant Meaning for 'virtual' (r = {r+1})", y=1.01, fontweight="bold")
plt.savefig(f"../results/permutation_testing/prt_pvalues_{year_start-1}_{year_end}_r={r+1}.png", bbox_inches="tight")
plt.show()

### **Benjamini Hochberg**

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

prt.plot(ax = ax, lw=2.2)
ax.legend(["PRT"], loc="upper left", fontsize=fs)
ax.set_ylabel("PRT", fontsize=fs+2)

ax2 = ax.twinx()
ax2.grid(visible=False)
ax2.set_xlim(1920)
ax2.set_ylabel("p-value", fontsize=fs+2, color=sns.color_palette()[3])
ax2.tick_params(axis='y', colors=sns.color_palette()[3])

ax2.scatter([x for x in range(year_start, year_end)], p_values, color=sns.color_palette("bright")[3], lw=1, label="p-values original")
ax2.scatter([x for x in range(year_start, year_end)], stats.false_discovery_control(p_values, method="bh"), color=sns.color_palette("bright")[2], lw=1, label="p-values Benjamini-Hochberg")
ax2.axhline(0.05, color=sns.color_palette()[3], ls="--")
ax2.annotate("significance threshold (0.05)", (1920, 0.05), xytext=(1920, 0.061),
            fontsize=fs, color=sns.color_palette()[3], ha='left', va='bottom')


plt.title(f"P-values and False Discovery Rate (adjusted with Benjamini-Hochberg procedure, r = {r+1})", y=1.01, fontweight="bold", fontsize=fs+2)
plt.legend(fontsize=fs)
plt.savefig(f"../results/permutation_testing/prt_pvalues_FDR_{year_start-1}_{year_end}_r={r+1}.png", bbox_inches="tight")
plt.show()